## Welcome to the (Un)Official KoboldCpp Colab Notebook
It's really easy to get started. Just press the two **Play** buttons below, and then connect to the **Cloudflare URL** shown at the end.

You can select a model from the dropdown or enter a HF model name or URL into the Model feild, formatting should look like `KoboldAI/LLaMA2-13B-Tiefighter-GGUF`, or `https://huggingface.co/TheBloke/MLewd-ReMM-L2-Chat-20B-Inverted-GGUF`.

If a custom HF model name fails to load you may want to dircetly enter the link of the desired model. Formating for a manual URL should look like `https://huggingface.co/KoboldAI/LLaMA2-13B-Tiefighter-GGUF/resolve/main/LLaMA2-13B-Tiefighter.Q4_K_M.gguf` and is obtained by copying the link adress from the download arrow of the desired quant, doing this will use the quant value from the link instead of from the quant dropdown.

For **13B** models The following configurations are tested and known to work
- `Q4_K_M / 4096 context` Standard configuration
- `Q4_K_M / 6144 context` Increased context with a small cost to quality and speed
- `Q5_K_M / 4096 context` Increased quality at a small cost to speed

For **7B** models The following configurations are tested and known to work
- `Q5_K_M / 4096 context` Standard configuration
- `Q5_K_M / 8192 context` Greatly increased context with a small cost to quality and speed
- `Q5_K_M / 16384 context` Will load, may suffer quality loss

For **11B** models The following configurations are tested and known to work.
- `Q5_K_M / 8192 context` Standard configuration
- `Q5_K_M / 12288 context` This should also work
- `Q5_K_M / 16384 context` Will load, may suffer quality loss


In [ ]:
#@title <-- Tap this if you play on Mobile { display-mode: "form" }
%%html
<b>Press play on the music player to keep the tab alive, then start KoboldCpp below</b><br/>
<audio src="https://raw.githubusercontent.com/KoboldAI/KoboldAI-Client/main/colab/silence.m4a" controls>

In [ ]:
#@title <b>v-- Enter your model below and then click this to start Koboldcpp</b>

Model = "KoboldAI/LLaMA2-13B-Tiefighter-GGUF" #@param ["KoboldAI/LLaMA2-13B-Tiefighter-GGUF","TheBloke/ReMM-SLERP-L2-13B-GGUF","TheBloke/Noromaid-13B-v0.1.1-GGUF","TheBloke/MythoMax-L2-13B-GGUF","TheBloke/Xwin-LM-13B-v0.2-GGUF","TheBloke/Stheno-L2-13B-GGUF","KoboldAI/LLaMA2-13B-Psyfighter2-GGUF","TheBloke/OpenHermes-2.5-Mistral-7B-GGUF","https://huggingface.co/Sao10K/Fimbulvetr-11B-v2-GGUF/resolve/main/Fimbulvetr-11B-v2-Test-14.q5_K_M.gguf"]{allow-input: true}
Quant = "Q4_K_M" #@param ["Q4_K_S","Q4_K_M","Q5_K_M"] {allow-input: true}
ContextSize = "4096" #@param [2048,4096,6144,8192,12288] {allow-input: true}

LocalTunnelBackup = False #@param {type:"boolean"}
ForceRebuild = False #@param {type:"boolean"}

#@markdown <hr>
LoadLLaVAmmproj = False #@param {type:"boolean"}
LLaVAmmproj = "https://huggingface.co/koboldcpp/mmproj/resolve/main/llama-13b-mmproj-v1.5.Q4_1.gguf" #@param ["https://huggingface.co/koboldcpp/mmproj/resolve/main/llama-13b-mmproj-v1.5.Q4_1.gguf","https://huggingface.co/koboldcpp/mmproj/resolve/main/mistral-7b-mmproj-v1.5-Q4_1.gguf","https://huggingface.co/koboldcpp/mmproj/resolve/main/llama-7b-mmproj-v1.5-Q4_0.gguf"]{allow-input: true}
VCommand = ""
#@markdown <hr>
LoadImgModel = False #@param {type:"boolean"}
ImgModel = "https://huggingface.co/koboldcpp/imgmodel/resolve/main/imgmodel_older_q4_0.gguf" #@param ["https://huggingface.co/koboldcpp/imgmodel/resolve/main/imgmodel_older_q4_0.gguf"]{allow-input: true}
SCommand = ""

import os
if not os.path.isfile("/opt/bin/nvidia-smi"):
  raise RuntimeError("⚠️Colab did not give you a GPU due to usage limits, this can take a few hours before they let you back in. Check out https://lite.koboldai.net for a free alternative (that does not provide an API link but can load KoboldAI saves and chat cards) or subscribe to Colab Pro for immediate access.⚠️")
if LLaVAmmproj and LoadLLaVAmmproj:
  VCommand = "--mmproj vmodel.gguf"
else:
  SCommand = ""
if ImgModel and LoadImgModel:
  SCommand = "--sdconfig imodel.gguf clamped 4 quant"
else:
  SCommand = ""


# Split the Model string into MODELPATH and MODELNAME
# Check if the Model string contains "huggingface.co"
if "huggingface.co" in Model:
    # Extract the path after "huggingface.co/"
    parts = Model.split("huggingface.co/")[1].split('/')
else:
    # Assume it's the short version and split the Model string
    parts = Model.split('/')
MODELPATH = parts[0]
MODELNAME = parts[1].replace('-GGUF', '')

print("MODELPATH:", MODELPATH)
print("MODELNAME:", MODELNAME)

if ForceRebuild:
  FLAG2 = "F"
else:
  FLAG2 = "concedo"



if "gguf" not in Model.lower() and "-ggml" not in Model.lower():
    raise ValueError("Error: The Model field does not contain the required string '-GGUF'. Please ensure that your selected model is in GGUF format and try again. Consider searching Huggingface with MODELNAME GGUF to find the desired model")
else:
    print("Link check valid.")

if ("ggml" in Model and ".bin" in Model.lower()):
    print("You appear to be loading a GGML file, GGUF has replaced GGML. This notebook will work, but some options, such as automatic scaling for extended context, will not work")
    !sleep 10
MODELURL = "https://huggingface.co/{MODELPATH}/{MODELNAME}-GGUF/resolve/main/{MODELNAME}.{Quant}.gguf"

Model = Model.replace("blob", "resolve")

if ".gguf" in Model:
  MODELURL = Model


%cd /content
!git clone https://github.com/LostRuins/koboldcpp
%cd /content/koboldcpp
kvers = !(cat koboldcpp.py | grep 'KcppVersion = ' | cut -d '"' -f2)
kvers = kvers[0]
!echo Finding prebuilt binary for {kvers}
!wget -O dlfile.tmp https://kcppcolab.{FLAG2}.workers.dev/?{kvers} && mv dlfile.tmp koboldcpp_cublas.so
!test -f koboldcpp_cublas.so && echo Prebuilt Binary Exists || echo Prebuilt Binary Does Not Exist
!test -f koboldcpp_cublas.so && echo Build Skipped || make koboldcpp_cublas LLAMA_CUBLAS=1 LLAMA_COLAB=1
!cp koboldcpp_cublas.so koboldcpp_cublas.dat
!apt install aria2 -y
!rm model.ggml
if ".gguf" in Model or ".bin" in Model:
  !aria2c -x 10 -o model.ggml --summary-interval=5 --download-result=default --allow-overwrite=true --file-allocation=none $Model
else:
  !aria2c -x 10 -o model.ggml --summary-interval=5 --download-result=default --allow-overwrite=true --file-allocation=none https://huggingface.co/{MODELPATH}/{MODELNAME}-GGUF/resolve/main/{MODELNAME}.{Quant}.gguf

if VCommand:
  !aria2c -x 10 -o vmodel.gguf --summary-interval=5 --download-result=default --allow-overwrite=true --file-allocation=none $LLaVAmmproj
if SCommand:
  !aria2c -x 10 -o imodel.gguf --summary-interval=5 --download-result=default --allow-overwrite=true --file-allocation=none $ImgModel


MODELNAME = MODELNAME.lower()
import os


folder_path = '/content/koboldcpp'
file_name = 'model.ggml'

full_path = os.path.join(folder_path, file_name)

if os.path.exists(full_path):
    print(f"The file {file_name} is present in the folder. Not redownloading")
else:
    print(f"The file {file_name} is not present in the folder.")
    !aria2c -x 10 -o model.ggml --summary-interval=5 --download-result=default --allow-overwrite=true --file-allocation=none  https://huggingface.co/{MODELPATH}/{MODELNAME}-GGUF/resolve/main/{MODELNAME}.{Quant}.gguf

if os.path.exists(full_path):
    print(f"The file {file_name} is present in the folder. Continuing")
else:
    print(f"The file {file_name} is not present in the folder.")
    print(f"The file is not present in the folder. All attempts to download the file based on entered Model name have failed. Please use the direct link method instead")
    !sleep 120
if LocalTunnelBackup:
  !npm install -g localtunnel
  !nohup lt --port 5001 &
  !sleep 8
  !cat nohup.out


if LocalTunnelBackup:
  !python koboldcpp.py model.ggml --usecublas 0 --multiuser --gpulayers 99 --contextsize $ContextSize $VCommand $SCommand --hordeconfig $MODELNAME 1 1 --onready "curl -s ipecho.net/plain | xargs -I % echo If using the Localtunnel backup scroll up for your Localtunnel link, then use the following IP to verify: %"
else:
  !python koboldcpp.py model.ggml --usecublas 0 --multiuser --gpulayers 99 --contextsize $ContextSize --hordeconfig $MODELNAME 1 1 --remotetunnel $VCommand $SCommand